## Here I translate GCN in tensorflow to GCN in tensorflow.keras


In [2]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras import layers
import conv_models
import tensorflow.keras.backend as K

In [3]:
# tf and tf keras versions
print(tf.VERSION)
print(tf.keras.__version__)

1.13.1
2.2.4-tf


In [4]:
# load the input data and test set

adj = np.load('/Users/b_eebs/tf-keras/ZINC/adj/0.npy').astype('float32')
features = np.load('/Users/b_eebs/tf-keras/ZINC/features/0.npy').astype('float32')
test = (np.load('/Users/b_eebs/tf-keras/ZINC/logP.npy')[0:10000]).astype('float32') #takes the first 10,000 molecules

In [7]:
class convLayer(layers.Layer):

  def __init__(self, output_dim, batch_size, **kwargs):
    self.output_dim = output_dim
    self.batch_size = batch_size # hyper parameter 
    super(convLayer, self).__init__(**kwargs)

  def build(self, input_shape):
    shape = tf.TensorShape((input_shape[1], self.output_dim))
    shape = [int(shape[0]),int(shape[1])] # [50 , 32]

    self.kernel = self.add_weight(name='kernel',
                                  shape=shape,
                                  initializer='glorot_uniform',
                                  trainable=True,dtype=tf.float32)

    self.bias = self.add_weight(name='bias',
                        shape=[shape[1]],
                        initializer='glorot_uniform',
                        trainable=True,dtype=tf.float32)

    super(convLayer, self).build(input_shape)

  def call(self, inputs):
    feat = inputs[0]
    adj = inputs[1]
    _features = conv_models.GCN(feat,adj,self.kernel,self.bias) #[ batch_size, 50, 32]
    _features = tf.reshape(_features, [batch_size, -1]) # [batch_size 1600], fl. 32
    _features = tf.cast(_features, tf.float64) # [batch_size 1600], fl. 64
    return _features

'''
  def compute_output_shape(self, input_shape):
    shape = tf.TensorShape(input_shape).as_list()
    shape[-1] = self.output_dim
    return tf.TensorShape(shape)

  def get_config(self):
    base_config = super(convLayer, self).get_config()
    base_config['output_dim'] = self.output_dim
    return base_config

  @classmethod
  def from_config(cls, config):
    return cls(**config)
'''

"\n  def compute_output_shape(self, input_shape):\n    shape = tf.TensorShape(input_shape).as_list()\n    shape[-1] = self.output_dim\n    return tf.TensorShape(shape)\n\n  def get_config(self):\n    base_config = super(convLayer, self).get_config()\n    base_config['output_dim'] = self.output_dim\n    return base_config\n\n  @classmethod\n  def from_config(cls, config):\n    return cls(**config)\n"

In [8]:
conv_output = 32
K.set_learning_phase(1)
batch_size = 100

# define the model, composed of one layer defined in convLayers class
model = tf.keras.Sequential([convLayer(32,batch_size), 
                            layers.Activation('relu')])

# model compiler that optimizes and monters the mse and mae per batch
model.compile(optimizer=tf.keras.optimizers.Adam(lr = 0.001),
              loss='mse',
              metrics=['mae'])


# Took 100 molecules and trained over 3 epochs with batch size = 100. Weird thing is,  number of molecules
# and the batch_size have to be the same or else it wont work. Look into this. ALSO will fail if there are no
# batches.
dataset = [features[:100],adj[0:100]]
model.fit(dataset, test[:100], batch_size = batch_size, epochs = 3)
model.summary()

AttributeError: 'list' object has no attribute 'shape'

In [3]:
# Here is my attempt at implementing 6 conv layers.
# I think what's happening is that the output from the first layer is directly going into the second layer
# flat. I need to figure out how to reshape the tensors after so that they can serve as usefule input
# into sequential layers. Confirm, this is happening by simply printing the input_shape to build method.

conv_output = 32
K.set_learning_phase(1)
batch_size = 100
num_layers = 6
model = tf.keras.Sequential()
for i in range(num_layers):
    model.add(convLayer(32,adj[:100],batch_size))
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='mse',
              metrics=['mae'])
model.fit(features[0:100], test[0:100],batch_size = 100)

NameError: name 'K' is not defined